In [ ]:
# Imports
import glob
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import seaborn as sns
import statsmodels.api as sm
import shutil
import csv
import yaml
from scipy import stats
import math
import sys
import datetime as dt


In [ ]:
dir = os.path.abspath('')
dir1 = os.path.dirname(dir)
if not dir1 in sys.path:
    sys.path.append(dir1)

from df_loader import main
from df_loader import METRICS as COL_NAMES


In [ ]:
def prepareFolder(path):
    if os.path.exists(path+'/') == True:
        shutil.rmtree(path)
    if os.path.exists(path+'/') == False:
        os.makedirs(path+'/')


In [ ]:
from df_loader import main
print(os.getcwd())
dfs, names = main()
# sanity check
for (df, name) in zip(dfs, names):
    print(df['usr'].iloc[0], name)

prepareFolder('plots/timeseries')


## Healthy data set

In [ ]:
# 00 healthy
plt.style.use('classic')

plt.rc('font', size=10)  # controls default text size
plt.rc('axes', titlesize=26)  # fontsize of the title
plt.rc('xtick', labelsize=10)  # fontsize of the x tick labels
plt.rc('ytick', labelsize=10)  # fontsize of the y tick labels
plt.rc('legend', fontsize=14)  # fontsize of the legend

df = dfs[0]
name = names[0]
index = 0

if not os.path.exists('plots/timeseries/'+name):
    os.mkdir('plots/timeseries/'+name)
else:
    shutil.rmtree('plots/timeseries/'+name)
    os.mkdir('plots/timeseries/'+name)


for column in df.columns:
    fig = plt.figure(figsize=(6.4, 4.8), dpi=600)
    df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]  # remove outliers

    # calculations
    df[column+'-avg-healthy'] = df[column].iloc[:].mean()
    df[column+'-rolling-avg'] = df[column].rolling(10).mean()
    df[column+'-rolling-non-overlapping-avg'] = df[column].rolling(10).mean()[
        10-1::10]
    df[column+'-rolling-non-overlapping-avg'].bfill(inplace=True)

    # first subplot
    ax1 = plt.subplot(311)
    ax1 = df[column].plot(color='C0',
                          alpha=1, linewidth=1, label=COL_NAMES[index].split(':')[0])
    plt.xlabel('')  # override

    upperBound = df[column].max()
    lowerBound = df[column].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax1 = plt.ylim(bottom=-1, top=1)
    else:
        ax1 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
                       
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right',
               borderaxespad=0, fancybox=True, framealpha=0.5)

    # second subplot
    ax2 = plt.subplot(312)
    ax2 = df[column+'-avg-healthy'].iloc[:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-rolling-non-overlapping-avg'].plot(
        color='C6', alpha=1, linewidth=1, label='rolling average')
    plt.xlabel('')  # override

    upperBound = df[column+'-rolling-non-overlapping-avg'].max()
    lowerBound = df[column+'-rolling-non-overlapping-avg'].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax2 = plt.ylim(bottom=-1, top=1)
    else:
        ax2 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)

    # third subplot
    ax3 = plt.subplot(313)
    # hacky solution
    ax3 = plt.axvline(df.index[120], color='C2',
                      linewidth=0)  # triggering malware
    ax3 = plt.axvline(df.index[240], color='C1',
                      linewidth=0, linestyle='solid')  # triggering MTD
    ax3 = df[column+'-avg-healthy'].iloc[:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='average')

    ax3.xaxis.set_major_locator(mdates.MinuteLocator())
    ax3.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

    
    upperBound = max([df[column+'-avg-healthy'].max()])
    lowerBound = min([df[column+'-avg-healthy'].min()])
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax3 = plt.ylim(bottom=-1, top=1)
    else:
        ax3 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))

    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=True)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    plt.xlabel('')  # override

    # legend
    fig.subplots_adjust(hspace=0.2, left=0.14, right=0.90)
    plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=True)
    lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
    plt.legend(lines[1:], labels[1:], bbox_to_anchor=(
        0.0, -0.5), loc='upper left', borderaxespad=0, fancybox=True, framealpha=0.5)

    plt.savefig('plots/timeseries/{}/{}-{}-timeseries.pdf'.format(name, name[3:], column,),
                facecolor='white', transparent=False, bbox_inches="tight")

    # shared legend
    figLegend = plt.figure(figsize=(5, 1.7))
    figLegend.legend(lines[1:], labels[1:], loc='upper left')
    figLegend.savefig(
        'plots/timeseries/{}/{}-timeseries-legend.pdf'.format(name, name[3:]))

    index += 1
    plt.cla()
plt.close('all')


In [ ]:
# 01 httpBackdoor
if not os.path.exists('plots/timeseries/'+name):
    os.mkdir('plots/timeseries/'+name)
plt.style.use('classic')

plt.rc('font', size=10)  # controls default text size
plt.rc('axes', titlesize=26)  # fontsize of the title
plt.rc('xtick', labelsize=10)  # fontsize of the x tick labels
plt.rc('ytick', labelsize=10)  # fontsize of the y tick labels
plt.rc('legend', fontsize=14)  # fontsize of the legend

df = dfs[1]
name = names[1]
index = 0

if not os.path.exists('plots/timeseries/'+name):
    os.mkdir('plots/timeseries/'+name)
index = 0


for column in df.columns:
    fig = plt.figure(figsize=(6.4, 4.8), dpi=600)
    df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]  # remove outliers

    # calculations
    df[column+'-avg-healthy'] = df[column].iloc[:120].mean()
    df[column+'-avg-infected'] = df[column].iloc[120:240].mean()
    df[column+'-avg-cleansed'] = df[column].iloc[240:].mean()
    df[column+'-rolling-avg'] = df[column].rolling(10).mean()
    df[column+'-rolling-non-overlapping-avg'] = df[column].rolling(10).mean()[
        10-1::10]
    df[column+'-rolling-non-overlapping-avg'].bfill(inplace=True)

    # first subplot
    ax1 = plt.subplot(311)
    ax1 = plt.axvline(df.index[120], color='C2',
                      linewidth=1.75)  # triggering malware
    ax1 = plt.axvline(df.index[240], color='C1',
                      linewidth=1.75, linestyle='solid')  # triggering MTD
    ax1 = df[column].plot(color='C0',
                          alpha=1, linewidth=1, label=COL_NAMES[index].split(':')[0])
    plt.xlabel('')  # override

    upperBound = df[column].max()
    lowerBound = df[column].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax1 = plt.ylim(bottom=-1, top=1)
    else:
        ax1 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right',
               borderaxespad=0, fancybox=True, framealpha=0.5)

    # second subplot
    ax2 = plt.subplot(312)
    ax2 = plt.axvline(df.index[120], color='C2', linewidth=2,
                      label='triggering malware httpBackdoor')  # triggering malware
    ax2 = plt.axvline(df.index[240], color='C1', linewidth=2,
                      linestyle='solid', label='triggering MTD2')  # triggering MTD
    ax2 = df[column+'-avg-healthy'].iloc[:120].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-infected'].iloc[120:240].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-cleansed'].iloc[240:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-rolling-non-overlapping-avg'].plot(
        color='C6', alpha=1, linewidth=1, label='rolling average')
    plt.xlabel('')  # override

    upperBound = df[column+'-rolling-non-overlapping-avg'].max()
    lowerBound = df[column+'-rolling-non-overlapping-avg'].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax2 = plt.ylim(bottom=-1, top=1)
    else:
        ax2 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))

    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)

    # third subplot
    ax3 = plt.subplot(313)
    ax3 = plt.axvline(df.index[120], color='C2',
                      linewidth=2)  # triggering malware
    ax3 = plt.axvline(df.index[240], color='C1',
                      linewidth=2, linestyle='solid')  # triggering MTD
    ax3 = df[column+'-avg-healthy'].iloc[:120].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='average per phase')
    ax3 = df[column+'-avg-infected'].iloc[120:240].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax3 = df[column+'-avg-cleansed'].iloc[240:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')

    ax3.xaxis.set_major_locator(mdates.MinuteLocator())
    ax3.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

    
    upperBound = max([df[column+'-avg-healthy'].max(),
                      df[column+'-avg-infected'].max(), df[column+'-avg-cleansed'].max()])
    lowerBound = min([df[column+'-avg-healthy'].min(),
                      df[column+'-avg-infected'].min(), df[column+'-avg-cleansed'].min()])
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax3 = plt.ylim(bottom=-1, top=1)
    else:
        ax3 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))

    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=True)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    plt.xlabel('')  # override
    
    # legend
    fig.subplots_adjust(hspace=0.2, left=0.14, right=0.90)
    plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=True)

    lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
    plt.legend(lines[1:], labels[1:], bbox_to_anchor=(
        0.0, -0.5), loc='upper left', borderaxespad=0, fancybox=True, framealpha=0.5)

    plt.savefig('plots/timeseries/{}/{}-{}-timeseries.pdf'.format(name, name[3:], column,),
                facecolor='white', transparent=False, bbox_inches="tight")

    # shared legend
    figLegend = plt.figure(figsize=(5, 1.7))
    figLegend.legend(lines[1:], labels[1:], loc='upper left')
    figLegend.savefig(
        'plots/timeseries/{}/{}-timeseries-legend.pdf'.format(name, name[3:]))
    index += 1
    plt.cla()
plt.close('all')


In [ ]:
# 02 backdoor
plt.style.use('classic')

plt.rc('font', size=10)  # controls default text size
plt.rc('axes', titlesize=26)  # fontsize of the title
plt.rc('xtick', labelsize=10)  # fontsize of the x tick labels
plt.rc('ytick', labelsize=10)  # fontsize of the y tick labels
plt.rc('legend', fontsize=14)  # fontsize of the legend

df = dfs[2]
name = names[2]
index = 0

if not os.path.exists('plots/timeseries/'+name):
    os.mkdir('plots/timeseries/'+name)


for column in df.columns:
    fig = plt.figure(figsize=(6.4, 4.8), dpi=600)
    df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]  # remove outliers

    # calculations
    df[column+'-avg-healthy'] = df[column].iloc[:120].mean()
    df[column+'-avg-infected'] = df[column].iloc[120:240].mean()
    df[column+'-avg-cleansed'] = df[column].iloc[240:].mean()
    df[column+'-rolling-avg'] = df[column].rolling(10).mean()
    df[column+'-rolling-non-overlapping-avg'] = df[column].rolling(10).mean()[
        10-1::10]
    df[column+'-rolling-non-overlapping-avg'].bfill(inplace=True)

    # first subplot
    ax1 = plt.subplot(311)
    ax1 = plt.axvline(df.index[120], color='C2',
                      linewidth=1.75)  # triggering malware
    ax1 = plt.axvline(df.index[240], color='C1',
                      linewidth=1.75, linestyle='solid')  # triggering MTD
    ax1 = df[column].plot(color='C0',
                          alpha=1, linewidth=1, label=COL_NAMES[index].split(':')[0])
    plt.xlabel('')  # override

    upperBound = df[column].max()
    lowerBound = df[column].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax1 = plt.ylim(bottom=-1, top=1)
    else:
        ax1 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)

    plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right',
               borderaxespad=0, fancybox=True, framealpha=0.5)

    # second subplot
    ax2 = plt.subplot(312)
    ax2 = plt.axvline(df.index[120], color='C2', linewidth=2,
                      label='triggering malware backdoor')  # triggering malware
    ax2 = plt.axvline(df.index[240], color='C1', linewidth=2,
                      linestyle='solid', label='triggering MTD2')  # triggering MTD
    ax2 = df[column+'-avg-healthy'].iloc[:120].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-infected'].iloc[120:240].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-cleansed'].iloc[240:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-rolling-non-overlapping-avg'].plot(
        color='C6', alpha=1, linewidth=1, label='rolling average')
    plt.xlabel('')  # override

    upperBound = df[column+'-rolling-non-overlapping-avg'].max()
    lowerBound = df[column+'-rolling-non-overlapping-avg'].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax2 = plt.ylim(bottom=-1, top=1)
    else:
        ax2 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))

    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)

    # third subplot
    ax3 = plt.subplot(313)
    ax3 = plt.axvline(df.index[120], color='C2',
                      linewidth=2)  # triggering malware
    ax3 = plt.axvline(df.index[240], color='C1',
                      linewidth=2, linestyle='solid')  # triggering MTD
    ax3 = df[column+'-avg-healthy'].iloc[:120].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='average per phase')
    ax3 = df[column+'-avg-infected'].iloc[120:240].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax3 = df[column+'-avg-cleansed'].iloc[240:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')

    ax3.xaxis.set_major_locator(mdates.MinuteLocator())
    ax3.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

    
    upperBound = max([df[column+'-avg-healthy'].max(),
                      df[column+'-avg-infected'].max(), df[column+'-avg-cleansed'].max()])
    lowerBound = min([df[column+'-avg-healthy'].min(),
                      df[column+'-avg-infected'].min(), df[column+'-avg-cleansed'].min()])
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax3 = plt.ylim(bottom=-1, top=1)
    else:
        ax3 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))

    
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=True)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    plt.xlabel('')  # override

    # legend
    fig.subplots_adjust(hspace=0.2, left=0.14, right=0.90)
    plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=True)
    lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
    plt.legend(lines[1:], labels[1:], bbox_to_anchor=(
        0.0, -0.5), loc='upper left', borderaxespad=0, fancybox=True, framealpha=0.5)

    plt.savefig('plots/timeseries/{}/{}-{}-timeseries.pdf'.format(name, name[3:], column,),
                facecolor='white', transparent=False, bbox_inches="tight")

    # shared legend
    figLegend = plt.figure(figsize=(5, 1.7))
    figLegend.legend(lines[1:], labels[1:], loc='upper left')
    figLegend.savefig(
        'plots/timeseries/{}/{}-timeseries-legend.pdf'.format(name, name[3:]))
    index += 1
    plt.cla()
plt.close('all')


In [ ]:
# 03 The Tick
plt.style.use('classic')

plt.rc('font', size=10)  # controls default text size
plt.rc('axes', titlesize=26)  # fontsize of the title
plt.rc('xtick', labelsize=10)  # fontsize of the x tick labels
plt.rc('ytick', labelsize=10)  # fontsize of the y tick labels
plt.rc('legend', fontsize=14)  # fontsize of the legend

df = dfs[3]
name = names[3]
index = 0

if not os.path.exists('plots/timeseries/'+name):
    os.mkdir('plots/timeseries/'+name)


for column in df.columns:
    fig = plt.figure(figsize=(6.4, 4.8), dpi=600)
    df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]  # remove outliers

    # calculations
    df[column+'-avg-healthy'] = df[column].iloc[:120].mean()
    df[column+'-avg-infected'] = df[column].iloc[120:240].mean()
    df[column+'-avg-cleansed'] = df[column].iloc[240:].mean()
    df[column+'-rolling-avg'] = df[column].rolling(10).mean()
    df[column+'-rolling-non-overlapping-avg'] = df[column].rolling(10).mean()[
        10-1::10]
    df[column+'-rolling-non-overlapping-avg'].bfill(inplace=True)

    # first subplot
    ax1 = plt.subplot(311)
    ax1 = plt.axvline(df.index[120], color='C2',
                      linewidth=1.75)  # triggering malware
    ax1 = plt.axvline(df.index[240], color='C1',
                      linewidth=1.75, linestyle='solid')  # triggering MTD
    ax1 = df[column].plot(color='C0',
                          alpha=1, linewidth=1, label=COL_NAMES[index].split(':')[0])

    plt.xlabel('')  # override

    
    upperBound = df[column].max()
    lowerBound = df[column].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax1 = plt.ylim(bottom=-1, top=1)
    else:
        ax1 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))

    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right',
               borderaxespad=0, fancybox=True, framealpha=0.5)

    # second subplot
    ax2 = plt.subplot(312)
    ax2 = plt.axvline(df.index[120], color='C2', linewidth=2,
                      label='triggering malware The Tick')  # triggering malware
    ax2 = plt.axvline(df.index[240], color='C1', linewidth=2,
                      linestyle='solid', label='triggering MTD2')  # triggering MTD
    ax2 = df[column+'-avg-healthy'].iloc[:120].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-infected'].iloc[120:240].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-cleansed'].iloc[240:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-rolling-non-overlapping-avg'].plot(
        color='C6', alpha=1, linewidth=1, label='rolling average')
    plt.xlabel('')  # override

    upperBound = df[column+'-rolling-non-overlapping-avg'].max()
    lowerBound = df[column+'-rolling-non-overlapping-avg'].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax2 = plt.ylim(bottom=-1, top=1)
    else:
        ax2 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))

    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)

    # third subplot
    ax3 = plt.subplot(313)
    ax3 = plt.axvline(df.index[120], color='C2',
                      linewidth=2)  # triggering malware
    ax3 = plt.axvline(df.index[240], color='C1',
                      linewidth=2, linestyle='solid')  # triggering MTD
    ax3 = df[column+'-avg-healthy'].iloc[:120].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='average per phase')
    ax3 = df[column+'-avg-infected'].iloc[120:240].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax3 = df[column+'-avg-cleansed'].iloc[240:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')

    ax3.xaxis.set_major_locator(mdates.MinuteLocator())
    ax3.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

    
    upperBound = max([df[column+'-avg-healthy'].max(),
                      df[column+'-avg-infected'].max(), df[column+'-avg-cleansed'].max()])
    lowerBound = min([df[column+'-avg-healthy'].min(),
                      df[column+'-avg-infected'].min(), df[column+'-avg-cleansed'].min()])

    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax3 = plt.ylim(bottom=-1, top=1)
    else:
        ax3 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))

    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=True)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    plt.xlabel('')  # override

    # legend
    fig.subplots_adjust(hspace=0.2, left=0.14, right=0.90)
    plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=True)

    lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
    plt.legend(lines[1:], labels[1:], bbox_to_anchor=(
        0.0, -0.5), loc='upper left', borderaxespad=0, fancybox=True, framealpha=0.5)

    plt.savefig('plots/timeseries/{}/{}-{}-timeseries.pdf'.format(name, name[3:], column,),
                facecolor='white', transparent=False, bbox_inches="tight")

    # shared legend
    figLegend = plt.figure(figsize=(5, 1.7))
    figLegend.legend(lines[1:], labels[1:], loc='upper left')
    figLegend.savefig(
        'plots/timeseries/{}/{}-timeseries-legend.pdf'.format(name, name[3:]))
    index += 1
    plt.cla()
plt.close('all')


In [ ]:
# 04 BASHLITE
plt.style.use('classic')

plt.rc('font', size=10)  # controls default text size
plt.rc('axes', titlesize=26)  # fontsize of the title
plt.rc('xtick', labelsize=10)  # fontsize of the x tick labels
plt.rc('ytick', labelsize=10)  # fontsize of the y tick labels
plt.rc('legend', fontsize=14)  # fontsize of the legend

df = dfs[4]
name = names[4]
index = 0
if not os.path.exists('plots/timeseries/'+name):
    os.mkdir('plots/timeseries/'+name)



for column in df.columns:
    fig = plt.figure(figsize=(6.4, 4.8), dpi=600)
    df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]  # remove outliers

    # calculations
    df[column+'-avg-healthy'] = df[column].iloc[:120].mean()
    df[column+'-avg-infected'] = df[column].iloc[120:240].mean()
    df[column+'-avg-cleansed'] = df[column].iloc[240:].mean()
    df[column+'-rolling-avg'] = df[column].rolling(10).mean()
    df[column+'-rolling-non-overlapping-avg'] = df[column].rolling(10).mean()[
        10-1::10]
    df[column+'-rolling-non-overlapping-avg'].bfill(inplace=True)

    # first subplot
    ax1 = plt.subplot(311)
    ax1 = plt.axvline(df.index[120], color='C2',
                      linewidth=1.75)  # triggering malware
    ax1 = plt.axvline(df.index[240], color='C1',
                      linewidth=1.75, linestyle='solid')  # triggering MTD
    ax1 = df[column].plot(color='C0',
                          alpha=1, linewidth=1, label=COL_NAMES[index].split(':')[0])
    plt.xlabel('')  # override

    upperBound = df[column].max()
    lowerBound = df[column].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax1 = plt.ylim(bottom=-1, top=1)
    else:
        ax1 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))

    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)

    plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right',
               borderaxespad=0, fancybox=True, framealpha=0.5)

    # second subplot
    ax2 = plt.subplot(312)
    ax2 = plt.axvline(df.index[120], color='C2', linewidth=2,
                      label='triggering malware BASHLITE')  # triggering malware
    ax2 = plt.axvline(df.index[240], color='C1', linewidth=2,
                      linestyle='solid', label='triggering MTD2')  # triggering MTD
    ax2 = df[column+'-avg-healthy'].iloc[:120].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-infected'].iloc[120:240].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-cleansed'].iloc[240:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-rolling-non-overlapping-avg'].plot(
        color='C6', alpha=1, linewidth=1, label='rolling average')

    plt.xlabel('')  # override

    upperBound = df[column+'-rolling-non-overlapping-avg'].max()
    lowerBound = df[column+'-rolling-non-overlapping-avg'].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax2 = plt.ylim(bottom=-1, top=1)
    else:
        ax2 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    

    # third subplot
    ax3 = plt.subplot(313)
    ax3 = plt.axvline(df.index[120], color='C2',
                      linewidth=2)  # triggering malware
    ax3 = plt.axvline(df.index[240], color='C1',
                      linewidth=2, linestyle='solid')  # triggering MTD
    ax3 = df[column+'-avg-healthy'].iloc[:120].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='average per phase')
    ax3 = df[column+'-avg-infected'].iloc[120:240].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax3 = df[column+'-avg-cleansed'].iloc[240:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')

    ax3.xaxis.set_major_locator(mdates.MinuteLocator())
    ax3.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

    
    upperBound = max([df[column+'-avg-healthy'].max(),
                      df[column+'-avg-infected'].max(), df[column+'-avg-cleansed'].max()])
    lowerBound = min([df[column+'-avg-healthy'].min(),
                      df[column+'-avg-infected'].min(), df[column+'-avg-cleansed'].min()])

    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax3 = plt.ylim(bottom=-1, top=1)
    else:
        ax3 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))

    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=True)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    plt.xlabel('')  # override

    # legend
    fig.subplots_adjust(hspace=0.2, left=0.14, right=0.90)
    plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=True)
    
    lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
    plt.legend(lines[1:], labels[1:], bbox_to_anchor=(
        0.0, -0.5), loc='upper left', borderaxespad=0, fancybox=True, framealpha=0.5)

    plt.savefig('plots/timeseries/{}/{}-{}-timeseries.pdf'.format(name, name[3:], column,),
                facecolor='white', transparent=False, bbox_inches="tight")

    # shared legend
    figLegend = plt.figure(figsize=(5, 1.7))
    figLegend.legend(lines[1:], labels[1:], loc='upper left')
    figLegend.savefig(
        'plots/timeseries/{}/{}-timeseries-legend.pdf'.format(name, name[3:]))
    index += 1
    plt.cla()
plt.close('all')


In [ ]:
# 05 Ransomware-PoC
plt.style.use('classic')

plt.rc('font', size=10)  # controls default text size
plt.rc('axes', titlesize=26)  # fontsize of the title
plt.rc('xtick', labelsize=10)  # fontsize of the x tick labels
plt.rc('ytick', labelsize=10)  # fontsize of the y tick labels
plt.rc('legend', fontsize=14)  # fontsize of the legend

df = dfs[5]
name = names[5]
index = 0

if not os.path.exists('plots/timeseries/'+name):
    os.mkdir('plots/timeseries/'+name)


for column in df.columns:
    fig = plt.figure(figsize=(6.4, 4.8), dpi=600)
    df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]  # remove outliers

    # calculations
    df[column+'-avg-healthy'] = df[column].iloc[:120].mean()
    df[column+'-avg-infected'] = df[column].iloc[120:270].mean()
    df[column+'-avg-cleansed'] = df[column].iloc[270:].mean()
    df[column+'-rolling-avg'] = df[column].rolling(10).mean()
    df[column+'-rolling-non-overlapping-avg'] = df[column].rolling(10).mean()[
        10-1::10]
    df[column+'-rolling-non-overlapping-avg'].bfill(inplace=True)

    # first subplot
    ax1 = plt.subplot(311)
    ax1 = plt.axvline(df.index[120], color='C2',
                      linewidth=1.75)  # triggering malware
    ax1 = plt.axvline(df.index[130], color='C1',
                      linewidth=1.75, linestyle='dotted')  # triggering MTD
    ax2 = plt.axvline(df.index[270], color='C1',
                      linewidth=2, linestyle='dotted')  # finished MTD
    ax1 = df[column].plot(color='C0',
                          alpha=1, linewidth=1, label=COL_NAMES[index].split(':')[0])
    plt.xlabel('')  # override

    upperBound = df[column].max()
    lowerBound = df[column].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax1 = plt.ylim(bottom=-1, top=1)
    else:
        ax1 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)

    plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right',
               borderaxespad=0, fancybox=True, framealpha=0.5)

    # second subplot
    ax2 = plt.subplot(312)
    ax2 = plt.axvline(df.index[120], color='C2', linewidth=2,
                      label='triggering malware Ransomware-PoC')  # triggering malware
    ax2 = plt.axvline(df.index[130], color='C1', linewidth=2,
                      linestyle='dotted', label='triggering MTD1')  # triggering MTD
    ax2 = plt.axvline(df.index[270], color='C1', linewidth=2,
                      linestyle='dotted', label='finished MTD1')  # finished MTD
    ax2 = df[column+'-avg-healthy'].iloc[:120].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-infected'].iloc[120:270].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-cleansed'].iloc[270:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-rolling-non-overlapping-avg'].plot(
        color='C6', alpha=1, linewidth=1, label='rolling average')
    plt.xlabel('')  # override

    upperBound = df[column+'-rolling-non-overlapping-avg'].max()
    lowerBound = df[column+'-rolling-non-overlapping-avg'].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax2 = plt.ylim(bottom=-1, top=1)
    else:
        ax2 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    
    # third subplot
    ax3 = plt.subplot(313)
    ax3 = plt.axvline(df.index[120], color='C2',
                      linewidth=2)  # triggering malware
    ax3 = plt.axvline(df.index[130], color='C1',
                      linewidth=2, linestyle='dotted')  # triggering MTD
    ax3 = plt.axvline(df.index[270], color='C1',
                      linewidth=2, linestyle='dotted')  # finished MTD
    ax3 = df[column+'-avg-healthy'].iloc[:120].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='average per phase')
    ax3 = df[column+'-avg-infected'].iloc[120:270].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax3 = df[column+'-avg-cleansed'].iloc[270:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')

    ax3.xaxis.set_major_locator(mdates.MinuteLocator())
    ax3.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

    
    upperBound = max([df[column+'-avg-healthy'].max(),
                      df[column+'-avg-infected'].max(), df[column+'-avg-cleansed'].max()])
    lowerBound = min([df[column+'-avg-healthy'].min(),
                      df[column+'-avg-infected'].min(), df[column+'-avg-cleansed'].min()])

    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax3 = plt.ylim(bottom=-1, top=1)
    else:
        ax3 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))

    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=True)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    plt.xlabel('')  # override

    # legend
    fig.subplots_adjust(hspace=0.2, left=0.14, right=0.90)
    plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=True)

    lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
    plt.legend(lines[1:], labels[1:], bbox_to_anchor=(
        0.0, -0.5), loc='upper left', borderaxespad=0, fancybox=True, framealpha=0.5)

    plt.savefig('plots/timeseries/{}/{}-{}-timeseries.pdf'.format(name, name[3:], column,),
                facecolor='white', transparent=False, bbox_inches="tight")

    # shared legend
    figLegend = plt.figure(figsize=(5, 1.7))
    figLegend.legend(lines[1:], labels[1:], loc='upper left')
    figLegend.savefig(
        'plots/timeseries/{}/{}-timeseries-legend.pdf'.format(name, name[3:]))

    index += 1
    plt.cla()
plt.close('all')


In [ ]:
# 06 BEURK
plt.style.use('classic')

plt.rc('font', size=10)  # controls default text size
plt.rc('axes', titlesize=26)  # fontsize of the title
plt.rc('xtick', labelsize=10)  # fontsize of the x tick labels
plt.rc('ytick', labelsize=10)  # fontsize of the y tick labels
plt.rc('legend', fontsize=14)  # fontsize of the legend

df = dfs[6]
name = names[6]
index = 0

if not os.path.exists('plots/timeseries/'+name):
    os.mkdir('plots/timeseries/'+name)


for column in df.columns:
    fig = plt.figure(figsize=(6.4, 4.8), dpi=600)
    df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]  # remove outliers

    # calculations
    df[column+'-avg-healthy'] = df[column].iloc[:120].mean()
    df[column+'-avg-infected'] = df[column].iloc[120:240].mean()
    df[column+'-avg-cleansed'] = df[column].iloc[240:].mean()
    df[column+'-rolling-avg'] = df[column].rolling(10).mean()
    df[column+'-rolling-non-overlapping-avg'] = df[column].rolling(10).mean()[
        10-1::10]
    df[column+'-rolling-non-overlapping-avg'].bfill(inplace=True)

    # first subplot
    ax1 = plt.subplot(311)
    ax1 = plt.axvline(df.index[120], color='C2',
                      linewidth=1.75)  # triggering malware
    ax1 = plt.axvline(df.index[240], color='C1',
                      linewidth=1.75, linestyle='dashed')  # triggering MTD
    ax1 = df[column].plot(color='C0',
                          alpha=1, linewidth=1, label=COL_NAMES[index].split(':')[0])
    plt.xlabel('')  # override

    upperBound = df[column].max()
    lowerBound = df[column].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax1 = plt.ylim(bottom=-1, top=1)
    else:
        ax1 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)

    plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right',
               borderaxespad=0, fancybox=True, framealpha=0.5)

    # second subplot
    ax2 = plt.subplot(312)
    ax2 = plt.axvline(df.index[120], color='C2', linewidth=2,
                      label='triggering malware BEURK')  # triggering malware
    ax2 = plt.axvline(df.index[240], color='C1', linewidth=2,
                      linestyle='dashed', label='triggering MTD3')  # triggering MTD
    ax2 = df[column+'-avg-healthy'].iloc[:120].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-infected'].iloc[120:240].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-cleansed'].iloc[240:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-rolling-non-overlapping-avg'].plot(
        color='C6', alpha=1, linewidth=1, label='rolling average')
    plt.xlabel('')  # override

    upperBound = df[column+'-rolling-non-overlapping-avg'].max()
    lowerBound = df[column+'-rolling-non-overlapping-avg'].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax2 = plt.ylim(bottom=-1, top=1)
    else:
        ax2 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    
    # third subplot
    ax3 = plt.subplot(313)
    ax3 = plt.axvline(df.index[120], color='C2',
                      linewidth=2)  # triggering malware
    ax3 = plt.axvline(df.index[240], color='C1',
                      linewidth=2, linestyle='dashed')  # triggering MTD
    ax3 = df[column+'-avg-healthy'].iloc[:120].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='average per phase')
    ax3 = df[column+'-avg-infected'].iloc[120:240].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax3 = df[column+'-avg-cleansed'].iloc[240:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')

    ax3.xaxis.set_major_locator(mdates.MinuteLocator())
    ax3.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

    
    upperBound = max([df[column+'-avg-healthy'].max(),
                      df[column+'-avg-infected'].max(), df[column+'-avg-cleansed'].max()])
    lowerBound = min([df[column+'-avg-healthy'].min(),
                      df[column+'-avg-infected'].min(), df[column+'-avg-cleansed'].min()])

    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax3 = plt.ylim(bottom=-1, top=1)
    else:
        ax3 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))

    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=True)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    plt.xlabel('')  # override

    # legend
    fig.subplots_adjust(hspace=0.2, left=0.14, right=0.90)
    plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=True)

    lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
    plt.legend(lines[1:], labels[1:], bbox_to_anchor=(
        0.0, -0.5), loc='upper left', borderaxespad=0, fancybox=True, framealpha=0.5)

    plt.savefig('plots/timeseries/{}/{}-{}-timeseries.pdf'.format(name, name[3:], column,),
                facecolor='white', transparent=False, bbox_inches="tight")

    # shared legend
    figLegend = plt.figure(figsize=(5, 1.7))
    figLegend.legend(lines[1:], labels[1:], loc='upper left')
    figLegend.savefig(
        'plots/timeseries/{}/{}-timeseries-legend.pdf'.format(name, name[3:]))

    index += 1
    plt.cla()
plt.close('all')


## bdvl

In [ ]:
# 07 bdvl
plt.style.use('classic')

plt.rc('font', size=10)  # controls default text size
plt.rc('axes', titlesize=26)  # fontsize of the title
plt.rc('xtick', labelsize=10)  # fontsize of the x tick labels
plt.rc('ytick', labelsize=10)  # fontsize of the y tick labels
plt.rc('legend', fontsize=14)  # fontsize of the legend

df = dfs[7]
name = names[7]
index = 0

if not os.path.exists('plots/timeseries/'+name):
    os.mkdir('plots/timeseries/'+name)


for column in df.columns:
    fig = plt.figure(figsize=(6.4, 4.8), dpi=600)
    df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]  # remove outliers

    # calculations
    df[column+'-avg-healthy'] = df[column].iloc[:120].mean()
    df[column+'-avg-infected'] = df[column].iloc[120:240].mean()
    df[column+'-avg-cleansed'] = df[column].iloc[240:].mean()
    df[column+'-rolling-avg'] = df[column].rolling(10).mean()
    df[column+'-rolling-non-overlapping-avg'] = df[column].rolling(10).mean()[
        10-1::10]
    df[column+'-rolling-non-overlapping-avg'].bfill(inplace=True)

    # first subplot
    ax1 = plt.subplot(311)
    ax1 = plt.axvline(df.index[120], color='C2',
                      linewidth=1.75)  # triggering malware
    ax1 = plt.axvline(df.index[240], color='C1',
                      linewidth=1.75, linestyle='dashed')  # triggering MTD
    ax1 = df[column].plot(color='C0',
                          alpha=1, linewidth=1, label=COL_NAMES[index].split(':')[0])
    plt.xlabel('')  # override

    upperBound = df[column].max()
    lowerBound = df[column].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax1 = plt.ylim(bottom=-1, top=1)
    else:
        ax1 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
   

    plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right',
               borderaxespad=0, fancybox=True, framealpha=0.5)

    # second subplot
    ax2 = plt.subplot(312)
    ax2 = plt.axvline(df.index[120], color='C2', linewidth=2,
                      label='triggering malware bdvl')  # triggering malware
    ax2 = plt.axvline(df.index[240], color='C1', linewidth=2,
                      linestyle='dashed', label='triggering MTD3')  # triggering MTD
    ax2 = df[column+'-avg-healthy'].iloc[:120].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-infected'].iloc[120:240].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-cleansed'].iloc[240:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-rolling-non-overlapping-avg'].plot(
        color='C6', alpha=1, linewidth=1, label='rolling average')
    plt.xlabel('')  # override

    upperBound = df[column+'-rolling-non-overlapping-avg'].max()
    lowerBound = df[column+'-rolling-non-overlapping-avg'].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax2 = plt.ylim(bottom=-1, top=1)
    else:
        ax2 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    
    # third subplot 
    ax3 = plt.subplot(313)
    ax3 = plt.axvline(df.index[120], color='C2',
                      linewidth=2)  # triggering malware
    ax3 = plt.axvline(df.index[240], color='C1',
                      linewidth=2, linestyle='dashed')  # triggering MTD
    ax3 = df[column+'-avg-healthy'].iloc[:120].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='average per phase')
    ax3 = df[column+'-avg-infected'].iloc[120:240].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax3 = df[column+'-avg-cleansed'].iloc[240:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')

    ax3.xaxis.set_major_locator(mdates.MinuteLocator())
    ax3.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

    upperBound = max([df[column+'-avg-healthy'].max(),
                      df[column+'-avg-infected'].max(), df[column+'-avg-cleansed'].max()])
    lowerBound = min([df[column+'-avg-healthy'].min(),
                      df[column+'-avg-infected'].min(), df[column+'-avg-cleansed'].min()])

    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax3 = plt.ylim(bottom=-1, top=1)
    else:
        ax3 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))

    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=True)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    plt.xlabel('')  # override

    # legend
    fig.subplots_adjust(hspace=0.2, left=0.14, right=0.90)
    plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=True)

    lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
    plt.legend(lines[1:], labels[1:], bbox_to_anchor=(
        0.0, -0.5), loc='upper left', borderaxespad=0, fancybox=True, framealpha=0.5)

    plt.savefig('plots/timeseries/{}/{}-{}-timeseries.pdf'.format(name, name[3:], column,),
                facecolor='white', transparent=False, bbox_inches="tight")

    # shared legend
    figLegend = plt.figure(figsize=(5, 1.7))
    figLegend.legend(lines[1:], labels[1:], loc='upper left')
    figLegend.savefig(
        'plots/timeseries/{}/{}-timeseries-legend.pdf'.format(name, name[3:]))

    index += 1
    plt.cla()
plt.close('all')
